# **Import and Review Datasets**

## Objectives

* 'Fetch data from Kaggle and save as raw data'

### Inputs

* [Ocean Trash Locator]()
* [House Price Predictor]()
* [Skin Checker]()
* [Disease Screener]()
* [Oil Slick Spread Predictor]()
* [Dog Emotions]()
* [Filter Maintenance](https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance)

### Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

### Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print('You set a new current directory')

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Fetch data from Kaggle

After importing your **kaggle.json** token file; run the following to recognize it in the session

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

We are using the following Kaggle URL: [https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance](https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance)

<!-- ![image.png](https://static.streamlit.io/examples/cat.jpg) -->
![image.png](/workspace/dataset-testing/static/img/PPM_Dataset_Kaggle.png)

Get the dataset path from the Kaggle url
* When you are viewing the dataset at Kaggle, check what is after '[https://www.kaggle.com/datasets/](https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance)' .

The following function: 
* Retrieves the Kaggle dataset
* Creates a destination folder folder for the data to be placed
* Downloads it to the destination folder
* Unzips the downloaded file
* Deletes the **.zip** file 
* Deletes unused copies of the data as MATLAB **.mat** files
* Removes any  **kaggle.json** files used to access the dataset on Kaggle

In [ ]:
KaggleDatasetPath = 'prognosticshse/preventive-to-predicitve-maintenance'
DestinationFolder = 'inputs/datasets/raw'   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
  && rm {DestinationFolder}/*.zip \
  && rm {DestinationFolder}/*.pdf \
  && rm {DestinationFolder}/*.mat \
#   && rm kaggle.json

---

# Load and Inspect Kaggle data

#### Load Data to Inspect

In [ ]:
import pandas as pd
df_test = pd.read_csv(f'inputs/datasets/raw/Test_Data_CSV.csv')
df_train = pd.read_csv(f'inputs/datasets/raw/Train_Data_CSV.csv')
# df_play = pd.read_csv(f'inputs/datasets/raw/Train_Data_CSV.csv')

#### Data Composition

Ideally, the split of these subsets from a single dataset would be:
* Training Set = 70-80% (to fit the model)
* Test Set = 20-30%
* Validation Set = 10-20% (cross validation, compare models and choose hyperparameters)


In [ ]:
df_train.to_numpy()
df_train.shape

In [ ]:
df_test.to_numpy()
df_test.shape

In [ ]:
df_test['Data_No'].count()

In [ ]:
train_size = float(df_train['Data_No'].count())
test_size = float(df_test['Data_No'].count())
print(f'Train Data Shape {df_train.shape}; is {(train_size / (train_size + test_size))*100:.2f}% of the total data')
print(f'Test Data Shape {df_test.shape}; is {(test_size / (train_size + test_size))*100:.2f}% of the total data')

In [ ]:
df_train.columns.to_list()

In [ ]:
df_train.Data_No.count()

In [ ]:
data_index = df_test['Data_No'].unique()
# df_train['Data_No'].nunique()
data_index

In [ ]:
data_index[0]

## Splitting the data
Easier to do add calculations df_test and df_train datasets first

In [ ]:
import random
random_list = list(df_test['Data_No'].unique())
random_test = random.sample(random_list, 20)
random_test

In [ ]:
grouped_by_sets = df_test.groupby(['Data_No'])
grouped_by_sets.get_group(23)


In [ ]:
row1 = df_test.loc[[4, 5, 6, 7]]
row1

In [ ]:
data = pd.read_csv("/workspace/dataset-testing/inputs/datasets/raw/Test_Data_CSV.csv", index_col ="Data_No")
df_test_rows = data.loc[15:20]
df_test_rows

In [ ]:
import random
data = pd.read_csv("/workspace/dataset-testing/inputs/datasets/raw/Test_Data_CSV.csv", index_col="Data_No")
# df_test_copy = pd.read_csv("/workspace/dataset-testing/inputs/datasets/raw/Train_Data_CSV.csv", index_col ="Data_No")
random_list = list(df_test['Data_No'].unique())
select_random = random.sample(random_list, 20)
df_validate = data.loc[select_random]
df_validate.shape
df_validate.sort_values('Data_No', ascending=True)
print(select_random)
print('------')
print('Randomized data to be extracted to db_validate:')
df_validate

In [ ]:
select_random.sort()
print('Data to be extracted to db_validate (in order):')
select_random

Shape of data BEFORE splitting Validation data from df_test

In [ ]:
df_test.shape

df_test data with df_validation data removed

In [ ]:
df_test_copy = df_test
df_test_less_validations = df_test_copy[~df_test_copy['Data_No'].isin(select_random)]
# print (df_test_less_validations)
df_test_less_validations

New training data shape

In [ ]:
df_test_less_validations.shape

Replace the indicies

In [ ]:
df_validate.reset_index(inplace=True, drop=False)
df_test_less_validations.reset_index(inplace=True, drop=True)
df_validate.shape
df_validate


In [ ]:
df_test_less_validations

List the shapes of the remaining data

In [ ]:
dfs = [df_train, df_test_less_validations, df_validate]

for df in dfs:
    print(df.shape)

#### Update Working Variables 

New df_test File

In [ ]:
df_test = df_test_less_validations
df_test

## Add Calculations

### Add Mass Calculations

* add single data: 
`dataframe[new_column] = 'Value'`

* add multiple data: `dataframe[new_column0, new_column1, new_column2] = [val1, val2, val3]`

Include numerical values fot Dust Type

In [ ]:
df_test['Dust_Density'] = [0.900 if n == 'ISO 12103-1, A2 Fine Test Dust' else (1.025 if n == 'ISO 12103-1, A3 Medium Test Dust' else 1.200) for n in df_test['Dust']]
df_test.head()

OR

In [ ]:
# def dust_in_grams_cm3(n):
#     if n == 'ISO 12103-1, A2 Fine Test Dust':
#        return 0.900
#     elif n == 'ISO 12103-1, A3 Medium Test Dust':
#        return 1.025
#     else:
#        return 1.200

# df_test['Dust_Density'] = df_test['Dust'].apply(dust_in_grams_cm3)
# df_test.head()

Mass per observation:

In [ ]:
# df_test['Mass_g'] = (df_test.Dust_feed/1000)*df_test.Dust_Density
df_test.loc[:,('Mass_g')] = (df_test.Dust_feed/1000)*df_test.Dust_Density
df_test.tail()

Cumulative Mass

In [ ]:
data = df_test.Data_No
df_test['Cumulative_Mass_g'] = df_test['Mass_g'].groupby(data).cumsum()
df_test.head()
df_test.loc[600:610]

Cumulative Time

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()

In [ ]:
# df_test['Ts'] = df_test['Time'].diff().fillna(df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time)
# df_test.head()

In [ ]:
# df_test['Ts'] = df_test['Time'].diff().fillna(df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time)
# df_test.loc[1211:1220]

### For Review
#### Correct for time Change

In [ ]:
# test = df_test.Data_No / df_test.Data_No.shift(1)
# # test.loc[1211:1220].squeeze()
# test.loc[1211:1220]

In [ ]:
# t_list = test.to_frame()
# t_list[1211:1220]

In [ ]:
# # test = (df_test.Data_No / df_test.Data_No.shift(1)).to_list()
# t_list = test.to_list()
# time_diff = df_test['Time'].diff().fillna(df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time)
# # time_rel = df_test[df_test.Data_No != df_test.Data_No.shift(1)]
# time_rel = df_test.Data_No.shift(1) - time_diff

# for i in test:
#     if i == 1:
#         # print('True')
#         df_test['Ts'] = time_diff
#     else:
#         # print('False')
#         df_test['Ts'] = time_rel
# # df_test.head()
# df_test.loc[1211:1222]

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()

In [ ]:
# first_row_time = df_test[df_test.Data_No != df_test.Data_No.shift(1)]
# first_row_time.Time.head()

In [ ]:
# test = df_test.Data_No / df_test.Data_No.shift(1)
# first_row_time = df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time
# # first_row_time
# for df_test['Ts'] in df_test:
#     if test.tolist() == 1:
#         df_test['Ts'] = df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time
#     else:
#         df_test['Ts'] = df_test['Time'].diff().fillna(df_test[df_test.Data_No != df_test.Data_No.shift(1)].Time)
#         # pass
    
# df_test.loc[365:370]

### Total Time Calculation

In [ ]:
import pandas as pd
time_total = df_test['Time'].groupby(data).max().to_frame()
time_total.index.name = None
time_total['Data_No'] = time_total.index
# time_total.reset_index(level=None, inplace=False).head()
# time_total_series = time_total.squeeze()
# time_total.to_frame()
# time_total_series
time_total.head()

In [ ]:
time_total.index.name

In [ ]:
# time_total.Time[3]

In [ ]:
time_total.index[0]

In [ ]:
time_total.index[1]

In [ ]:
type(int(time_total.index[1]))

In [ ]:
type(time_total.Data_No)

In [ ]:
type(time_total)

In [ ]:
# time_total = time_total.drop(axis=1, columns='index_no')
# time_total

#### To convert it into a series

In [ ]:
time_total.squeeze().head()

#### .Map() Testing

In [ ]:
time_total.Time.map('I am a {}'.format).head()

Convert Total Time Tt

In [ ]:
df_test['Total_Test_T'] = df_test['Data_No'].map(time_total.set_index('Data_No')['Time'])
df_test.tail()

In [ ]:
data = df_test.Data_No
df_test['Time'].groupby(data).max()[3]

In [ ]:
df_test.head()

In [ ]:
df_test.loc[365:370]

Convert time Interval Ts

Time Start

In [ ]:
# # import pandas as pd
# data = df_test.Data_No
# time_start = df_test['Time'].groupby(data).min().to_frame()
# time_start.index.name = None
# time_start['Data_No'] = time_start.index
# time_start.head()

In [ ]:
# df_test['Ts'] = df_test['Data_No'].map(time_start.set_index('Data_No')['Time'])
# df_test.loc[365:370]
# df_test.loc[1211:1220]

In [ ]:
# df_test['Data_No']

In [ ]:
# time_total.index

In [ ]:
# time_total.index.to_series()

### RUL

Remaining Useful Life (RUL) = Total time (cycles) to failure for each life test (T) - current time (t)

In [ ]:
df_test.loc[1210:1220]

In [ ]:
data = df_test.Data_No
RUL_end = df_test['RUL'].groupby(data).min().to_frame()
RUL_end.index.name = None
RUL_end['Data_No'] = RUL_end.index
RUL_end.head()

In [ ]:
# RUL_Start = df_test['Data_No'].map(RUL_end.set_index('Data_No')['RUL'])
# RUL_Start

In [ ]:
# data = df_test.Data_No
# RUL = (df_test['Total_Test_T'] - df_test['Time']) + RUL_Start
# RUL.loc[1210:1220]

In [ ]:
# data = df_test.Data_No
# RUL = (df_test['Total_Test_T'] - df_test['Time']) + RUL_Start
# RUL.tail()

#### RUL for df_test

In [ ]:
data = df_test.Data_No
RUL_Start = df_test['Data_No'].map(RUL_end.set_index('Data_No')['RUL'])
df_test['RUL_test'] = (df_test['Total_Test_T'] - df_test['Time']) + RUL_Start
df_test.loc[1210:1220]

#### RUL for df_train

In [ ]:
# data = df_train.Data_No
# RUL_Start = df_train['Data_No'].map(RUL_end.set_index('Data_No')['RUL'])
# df_train['RUL_test'] = (df_train['Total_Test_T'] - df_train['Time']) + RUL_Start
# df_train.loc[1210:1220]

### Investigate Data Splits

Create print variable name function

In [ ]:
def print_variable_name(data):
    name =[x for x in globals() if globals()[x] is data][0]
    print("%s"%name)

In [ ]:
dfs = [df_train, df_test, df_validate]

train = int(df_train.Data_No.count())
test = int(df_test.Data_No.count())
validation = int(df_validate.Data_No.count())
total = test + train + validation

for df in dfs:
    # print(f'{print_variable_name(df)}{df.shape} = {(int(df.Data_No.count())/total)*100:.2f}% split')
    print(f'{df.shape} = {(int(df.Data_No.count())/total)*100:.2f}% split')

In [ ]:
df_train.shape
# df_test.shape
# df_validate.shape
# df_test_less_validations.shape

### Save Copy of the Datasets for Engineering

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

# df.to_csv(f'outputs/datasets/collection/FilterMaintenancePredictorDataset.csv',index=False)
df_train.to_csv(f'outputs/datasets/collection/df_train.csv',index=False)
df_test.to_csv(f'outputs/datasets/collection/df_test.csv',index=False)
df_validate.to_csv(f'outputs/datasets/collection/df_validate.csv',index=False)

---

# Scrapbook

In [ ]:
# df_test.columns.to_list()

In [ ]:
# df_test_np = df_test.to_numpy()
# df_test_np

In [ ]:
# df_test['Data_No'].to_list()

In [ ]:
last_test_row = []
for col in df_test.columns.values:
    last_test_row.append(df_test[col].iloc[-1])
print(last_test_row)

### List of the observations at the end of each life test. 
Used to answer the question:
* **Did the filter fail at the end of the test**?
    * Will help us indicate if the test is part of the right censored test group

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(1)]

### Last Values in each Data Set

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)]

In [ ]:
# last_test_row = df_test[df_test.Data_No != df_test.Data_No.shift(-1)]['Time']
last_test_row = df_test[df_test.Data_No != df_test.Data_No.shift(-1)]
last_test_row.head()
# (last_test_row*10).head()


In [ ]:
# df_test['Total_Test_T'] = last_test_row.Time
# df_test

In [ ]:
# last_test_row.head()

In [ ]:
last_test_row.iloc[0]

In [ ]:
last_test_row.iloc[0]['Time']

In [ ]:
last_test_row.iloc[0]['Data_No']

In [ ]:
last_test_row.index + 1

To detect a change in the observation value at 'Data_No'

In [ ]:
arr2 = [2, 2, 2, 3, 3, 1, 2, 2, 9, 4, 4]
# index = arr2.index(item)

for n in range(len(arr2)):
    if arr2[n] == arr2[n-1]:
        print(f'Same values!')
    else:
        print(f'Value Change: Current = {arr2[n]} to Previous = {arr2[n-1]}!')

In [ ]:
df_test['Data_No'].tail()

In [ ]:
data = df_test['Data_No']
data


In [ ]:
df_test.Data_No[4454]

In [ ]:
data_index

In [ ]:
data_index[2]

In [ ]:
last_test_row.iloc[0]['Time']

In [ ]:
data = df_test['Data_No']
# index_ref = last_test_row.iloc[data.index]['Time']

for n in range(len(data)-1):
    if data[n] != data[n+1]:
        print(f'Value Change at index : {data.index[n+2]}')
        # data['Total_Test_T'] = last_test_row.iloc[df_test.Data_No[n-1]]['Time']

In [ ]:
# data = df_test['Data_No']
# # index_ref = last_test_row.iloc[data.index]['Time']

# for n in range(len(data)+1):
#     if data[n+1] != data[n]:
#         # print(f'Value Change: Current = {data[n]} to Next = {data[n+1]}! Current Index: {data.index[n]}')
#         # print(f'Value Change at index : {data.index[n+2]} to last_test_row.iloc["data"-1]["Time"]')
#         data[n+1] = last_test_row.iloc[df_test.Data_No[n]]['Time']

In [ ]:
# for n in df_test['Data_No']:
#     if df_test.Data_No[n] > df_test.Data_No[n-1]:
#         df_test['Total_Test_T'] = last_test_row.Time
# df_test

In [ ]:
# df_train['Data_No'].value_counts().unique()
# df_train['Data_No'].value_counts()
count = df_train.groupby(['Data_No']).count()
# print(count.head())
print(count)

In [ ]:
df_test.columns[6]

### Test Data
* includes the Remaining Useful Life (RUL) target variable sourced from live measures 

In [ ]:
df_test

Test DataFrame Summary

In [ ]:
df_test.info()
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.describe()

In [ ]:
df_test.corr()

### List of the observations at the end of each life test. 
Used to answer the question:
* **Did the filter fail at the end of the test**?
    * Will help us indicate if the test is part of the right censored test group

In [ ]:
# df_test('2').head()

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)]

### Train Data

In [ ]:
df_train

Train DataFrame Summary

In [ ]:
df_train.info()
df_train.head()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.describe()

In [ ]:
df_train.corr()

In [ ]:
df_train[df_train.Data_No != df_train.Data_No.shift(1)]

In [ ]:
df_train[df_train.Data_No != df_train.Data_No.shift(-1)]

---

# Push Files to Repo

Add RUL column to Train Data

In [ ]:
# df_train.insert(loc=6, column='RUL', value=0.0, allow_duplicates=False)
# df_train

In [ ]:
# df_train.dtypes['RUL']
# df_train.info()

#### Combine Files

In [ ]:
combined_list = [df_test, df_train, df_validate]
df = pd.concat(combined_list)
df

* In case you don't need to push files to Repo, you may replace this section with 'Conclusions and Next Steps' and state your conclusions and next steps.

In [ ]:
# pip install openpyxl

In [ ]:
# import os
# try:
#   os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
# except Exception as e:
#   print(e)

# df.to_csv(f'outputs/datasets/collection/df_total.csv',index=False)
# df_test.to_csv(f'outputs/datasets/collection/df_test.csv',index=False)
# df_train.to_csv(f'outputs/datasets/collection/df_train.csv',index=False)
# df_validate.to_csv(f'outputs/datasets/collection/df_validate.csv',index=False)

---

# Notes Section

#### Combine Train & Test Data

In [ ]:
# import pandas as pd
# df_test = pd.read_csv(f'inputs/datasets/raw/Test_Data_CSV.csv')
# df_train = pd.read_csv(f'inputs/datasets/raw/Train_Data_CSV.csv')
# combined_list = [df_test, df_train]
# df = pd.concat(combined_list)
# df

In [ ]:
df.info()
df.head()

In [ ]:
df[df.duplicated(subset=['Data_No'])]

## Impute Missing Remaining Useful Life (RUL) Data

* The RUL information of the test data is not an estimate, rather the actual time when the experiment exceeded the threshold. 
    * In order to define a specific test problem, the measurements in the test data set are right-censored at random points and only the corresponding RUL information is provided.

#### Test Overfitting

#### Test Underfitting

Check missing data:
* https://docs.google.com/document/d/1yXb5g5IU7IldBpND1FbIfHIyGGmNSlXogarAyBlNO_g/edit?usp=sharing
* df.isnull()
* df.isnull().sum()

In [ ]:
df.isnull()
df.isnull().sum()

You can calculate the value to be filled in. 
* The example below calculates the mean for column A and inserts this value where it is missing for that column.


In [ ]:
# df['RUL'].fillna(value=df['RUL'].mean(),inplace=True)
# df

## Manage .mat files in Python?

The data has been created using MATLAB as data.mat file.
* However the source contributor has uploaded the data as CSV files as well. 
* They indicate the although file structure is slightly different between the .csv the .mat filed, the variable names have been kept however.

#### Install scipy

In [ ]:
# pip install scipy

Import the scipy.io.loadmat module

In [ ]:
# from os.path import dirname, join as pjoin
# import scipy.io as sio

# data_dir = pjoin(dirname(sio.__file__), 'matlab', 'tests', 'data')
# mat_fname = pjoin(data_dir, '/workspace/dataset-testing/inputs/datasets/raw/Data.mat')
# mat_contents = sio.loadmat(mat_fname)
# sorted(mat_contents.keys())

In [ ]:
# mat_contents['None']

OR?

In [ ]:
# from scipy.io import loadmat
# annots_data = loadmat(f'inputs/datasets/raw/Data.mat')
# annots_fine = loadmat(f'inputs/datasets/raw/Particle size distribution_ISO_12103_1_A2_Fine.mat')
# annots_medium = loadmat(f'inputs/datasets/raw/Particle size distribution_ISO_12103_1_A3_Medium.mat')
# annots_coarse = loadmat(f'inputs/datasets/raw/Particle size distribution_ISO_12103_1_A4_Coarse.mat')
# print(annots_data)

In [ ]:
# df_test.join(
    
# )

In [ ]:
# # insert new column at index 8
# df_test.insert(
#     loc=7,
#     column='Dust_Density',
#     value=[i for i in range(1000, 2000)]
# )

---

## Section N

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

## Push files to Repo

#### Combine Files

In [ ]:
# combined_list = [df_test, df_train]
# df = pd.concat(combined_list)
# df

* In case you don't need to push files to Repo, you may replace this section with 'Conclusions and Next Steps' and state your conclusions and next steps.

In [ ]:
# import os
# try:
#   os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
# except Exception as e:
#   print(e)

# df.to_csv(f'outputs/datasets/collection/FilterMaintenancePredictorDataset.csv',index=False)
# df_test.to_csv(f'outputs/datasets/collection/Test_FilterMaintenancePredictorDataset.csv',index=False)
# df_train.to_csv(f'outputs/datasets/collection/Train_FilterMaintenancePredictorDataset.csv',index=False)